In [1]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
#from matplotlib_venn import venn2



In [6]:

file_path = 'data/test.json'

# Открываем файл и загружаем данные
with open(file_path, 'r') as file:
    data = json.load(file)


In [7]:
df = pd.DataFrame(data.values(), data.keys())
df = df.reset_index()
df = df.rename(columns = {'index': 'user_id'})

df

,user_id,features
0,user_155203,"{'visits': [{'site-id': 20, 'first-seen': 1696..."
1,user_155204,"{'orders': [{'site-id': 489, 'orders': [{'crea..."
2,user_155205,"{'visits': [{'site-id': 3, 'first-seen': 16971..."
3,user_155206,"{'orders': [{'site-id': 62, 'orders': [{'creat..."
4,user_155207,"{'orders': [{'site-id': 104, 'orders': [{'crea..."
...,...,...
18779,user_173982,"{'orders': [{'site-id': 21, 'orders': [{'creat..."
18780,user_173983,"{'orders': [{'site-id': 34, 'orders': [{'creat..."
18781,user_173984,"{'orders': [{'site-id': 29, 'orders': [{'creat..."
18782,user_173985,"{'orders': [{'site-id': 114, 'orders': [{'crea..."


In [8]:
#подготовим проверочный датасет для итоговой сверки:
# Извлечение числа из столбца user_id
df_final = df
df_final['user_id'] = df_final['user_id'].apply(lambda x: int(x.split('_')[1]))

# Преобразование столбца target
#df_final['target'] = df_final['target'].apply(lambda x: 1 if x == 'female' else 0)

df_final

,user_id,features
0,155203,"{'visits': [{'site-id': 20, 'first-seen': 1696..."
1,155204,"{'orders': [{'site-id': 489, 'orders': [{'crea..."
2,155205,"{'visits': [{'site-id': 3, 'first-seen': 16971..."
3,155206,"{'orders': [{'site-id': 62, 'orders': [{'creat..."
4,155207,"{'orders': [{'site-id': 104, 'orders': [{'crea..."
...,...,...
18779,173982,"{'orders': [{'site-id': 21, 'orders': [{'creat..."
18780,173983,"{'orders': [{'site-id': 34, 'orders': [{'creat..."
18781,173984,"{'orders': [{'site-id': 29, 'orders': [{'creat..."
18782,173985,"{'orders': [{'site-id': 114, 'orders': [{'crea..."


In [9]:
df_final = df_final.drop('features', axis=1)

In [10]:
df_final.to_parquet('data/fixed/test_final.parquet')

In [14]:


def process_dataframe(df):
    # Добавляем столбец 'length', содержащий количество элементов в словаре
    df['length'] = df['features'].apply(lambda x: sum(len(v) if isinstance(v, list) else 1 for v in x.values()))

    # Преобразуем столбец 'features' с помощью json_normalize
    df_filtered = df.join(pd.json_normalize(df['features'])).drop('features', axis='columns')

    return df_filtered

def explode_column(df, level_labels):
    for level in level_labels:
        df = df.explode(level, ignore_index=True)
        df = df.rename(columns={level: level+'_old'})
        df = df.join(pd.json_normalize(df[level+'_old'])).drop(level+'_old', axis='columns')
    return df


def remove_outliers(df):
    Q1 = df['length'].quantile(0.25)
    Q3 = df['length'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.2 * IQR
    upper_bound = Q3 + 1.2 * IQR
    return df[(df['length'] <= upper_bound)]

def with_outliers(df):
    Q1 = df['length'].quantile(0.25)
    Q3 = df['length'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.2 * IQR
    upper_bound = Q3 + 1.2 * IQR
    return df[(df['length'] > upper_bound)]

# Применяем первую часть кода к исходному DataFrame
df_filtered = process_dataframe(df)

 #(комментим эту часть для получения полного DF)Применяем функцию для каждой группы в 'target'
df_orders_flat = df_filtered.groupby('user_id', group_keys=False).apply(remove_outliers)
# Сбрасываем индексы, чтобы получить DataFrame аналогичный исходному
df_orders_flat = df_orders_flat.reset_index(drop=True)

 #Применяем функцию для каждой группы в 'target'
#df_orders_flat_out = df_filtered.groupby('user_id', group_keys=False).apply(with_outliers)
#df_orders_flat_out = df_orders_flat_out.reset_index(drop=True)
'''
 #Применяем функцию для каждой группы в 'target'
df_filtered = df_filtered.groupby('target', group_keys=False).apply(remove_outliers)

# Сбрасываем индексы, чтобы получить DataFrame аналогичный исходному
df_filtered = df_filtered.reset_index(drop=True)
'''
# Применяем функцию для каждой группы в 'target'
#df_filtered_out = df_filtered.groupby('target', group_keys=False).apply(with_outliers)
# Сбрасываем индексы, чтобы получить DataFrame аналогичный исходному
#df_filtered_out = df_filtered_out.reset_index(drop=True)
#df_filtered_out = with_outliers(df_filtered)
# Выбираем нужные столбцы для df_orders
df_orders = df_orders_flat[['user_id', 'orders', 'length']].copy()
#df_orders1 = df_orders_flat_out[['user_id', 'orders', 'length']].copy()
# Задаем метки уровней для функции explode_column
levels_labels = ['orders', 'orders', 'items']

# Применяем вторую часть кода к df_orders
df_orders_flat1 = explode_column(df_orders, levels_labels)



# Применяем вторую часть кода к df_orders
#df_orders_flat_out1 = explode_column(df_orders1, levels_labels)





In [13]:
df_orders_flat_out

,user_id,length,visits,site-meta,exchange-sessions,last-visits-in-categories,orders


In [15]:
df_orders_flat1

,user_id,length,site-id,created-at,id,count,general-category-path,brand-id
0,155203,17,NaN,NaN,NaN,NaN,NaN,NaN
1,155204,27,489.0,1.405923e+09,NaN,NaN,NaN,NaN
2,155205,3,NaN,NaN,NaN,NaN,NaN,NaN
3,155206,33,62.0,1.621174e+09,item_13288,1.0,"[91214, 91197, 90801]",NaN
4,155206,33,62.0,1.621174e+09,item_13288,1.0,"[91214, 91197, 90801]",NaN
...,...,...,...,...,...,...,...,...
535401,173986,189,79.0,1.563164e+09,item_6640458,1.0,"[7811908, 7811873, 7877999]",1750.0
535402,173986,189,79.0,1.563164e+09,item_6640459,1.0,"[7811908, 7811873, 7877999]",1750.0
535403,173986,189,79.0,1.563164e+09,item_6640460,1.0,"[7812151, 7811877, 7877999]",1750.0
535404,173986,189,79.0,1.628942e+09,NaN,NaN,NaN,NaN


In [16]:
df_orders_flat1.to_parquet('data/fixed/test_test.parquet')

In [17]:
df_orders_flat_out1.to_parquet('data/fixed/test_test_out.parquet')

NameError: name 'df_orders_flat_out1' is not defined